## TODOs
&mdash; ~~Document pipeline tool~~\
&mdash; Document preprocessor\
&mdash; Agent that examines instructions, documents, and chat history to help determine the best course of action\
&mdash; Front-end chat interface with a document import that connects to the document ingestion pipe\
&mdash; Front-end chat interface that let's you dictate the system prompt\
&mdash; Haystack Retreivers and Readers\
&mdash; Prompts for the various LLMs invokes\
&mdash; Add a button to add your system prompt\
&mdash; Add a button to upload your docs (pdf's)\

### ChatAI Prototype

In [3]:
# Prompts 

from haystack.nodes import PromptTemplate

"""
'zero-shot-react'
You are a helpful and knowledgeable agent. To achieve your goal of answering complex questions
correctly, you have access to the following tools:

{tool_names_with_descriptions}

To answer questions, you'll need to go through multiple steps involving step-by-step thinking and
selecting appropriate tools and their inputs; tools will respond with observations. When you are ready
for a final answer, respond with the `Final Answer:`

Use the following format:

Question: the question to be answered
Thought: Reason if you have the final answer. If yes, answer the question. If not, find out the missing information needed to answer it.
Tool: pick one of {tool_names} 
Tool Input: the input for the tool
Observation: the tool will respond with the result
...

Final Answer: the final answer to the question, make it short (1-5 words)
Thought, Tool, Tool Input, and Observation steps can be repeated multiple times, but sometimes we can find an answer in the first pass
---

Question: {query}
Thought: Let's think step-by-step, I first need to

"""

# # # # # # # # # # # # # # # # # # # # # 

"""
'conversational-agent-without-tools'
The following is a conversation between a human and an AI.\n{memory}\nHuman: {query}\nAI:

"""

# # # # # # # # # # # # # # # # # # # # # 

"""
'conversational-summary'
Condense the following chat transcript by shortening and summarizing the content 
without losing important information:\n{chat_transcript}\nCondensed Transcript:

"""


chatAgentPrompt = PromptTemplate(
    prompt = """
    The following is a converation between a user and a virtual assistant. Given the context, provide the next best clear and concise response to the user's last question in 100 words or less. If you are unable to answer the question, you can say "I'm sorry but I can't help with that". {test} Context: {memory}; 
    Question: {query}; 
    Answer:
    """
) 

chatSummaryPrompt = PromptTemplate(
    prompt = """
    Summarize the following chat transcipt focusing only the most important information in the questions and responses. Be clear and concise and DO NOT lose the most relevant parts of the conversation: \n\n{chat_transcript}\nCondensed Transcript:
    """
)

In [ ]:
# TODO Document pre-processor

# https://haystack.deepset.ai/tutorials/08_preprocessing

In [7]:
# Document Pipeline

# Chat with Documemts
# https://haystack.deepset.ai/tutorials/25_customizing_agent

# Chat with Embeddings
# https://haystack.deepset.ai/tutorials/23_answering_multihop_questions_with_agents

import os
from datasets import load_dataset
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser, BM25Retriever
from haystack.pipelines import Pipeline
from haystack.utils import print_answers

remote_dataset = load_dataset("Tuana/presidents", split="train")

document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(remote_dataset)

from haystack.nodes import EmbeddingRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    use_gpu=True
)

document_store.update_embeddings(retriever=retriever)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
presidents_qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

from haystack.utils import print_answers

# result = presidents_qa.run("Who was the 1st president of the USA?")
result = presidents_qa.run("What year was the 1st president of the USA born?")

print_answers(result, "minimum")

Updating BM25 representation...: 100%|██████████| 151/151 [00:00<00:00, 18859.51 docs/s]
The prompt has been truncated from 938 tokens to 924 tokens so that the prompt length and answer length (100 tokens) fit within the max token limit (1024 tokens). Shorten the prompt to prevent it from being cut off.


'Query: What does Rhodes Statue look like?'
'Answers:'
[   {   'answer': ' was the fate of the remains of the Colossus statue after '
                  'it was destroyed, and who may have used the metal from the '
                  'statue?\n'
                  '    Answer: The ultimate fate of the remains of the statue '
                  'is uncertain, but it is unlikely that further maintenance '
                  'or rebuilding was done on an ancient pagan statue after the '
                  'city was Christianized in the 4th century. The metal may '
                  'have been used for coins and tools during earlier conflicts '
                  'such as the Sassanian wars, and by the'}]
'Query: What does Taylor Swift look like?'
'Answers:'
[   {   'answer': ' Based solely on the given documents, it is not possible to '
                  'answer this question as the documents do not contain any '
                  "information about Taylor Swift's appearance."}]


In [ ]:
# Tools for AI Agent

from haystack.agents import Tool

search_tool = Tool(
    name="seven_wonders_search",
    pipeline_or_node=generative_pipeline,
    description="Useful for when you need to answer questions about the seven wonders of the world",
    output_variable="answers",
)

In [ ]:
# Chat AI Agent

import os
from haystack.nodes import PromptNode
from haystack.agents.conversational import ConversationalAgent
from haystack.agents.memory import ConversationSummaryMemory

llm = PromptNode(
    model_name_or_path = "HuggingFaceH4/zephyr-7b-beta",
    api_key = os.getenv("HF_TOKEN"),
    max_length = 256,
    model_kwargs = {
        "max_new_tokens": 1000,
        "context_length": 2500 
    }
    
)

chat_node = llm
chat_node.stop_words = ["\nHuman"]

agentQuery = ConversationalAgent(
    prompt_node = chat_node,
    prompt_template = chatAgentPrompt,
    max_steps = 2,
    memory = ConversationSummaryMemory(
        prompt_node = llm,
        # prompt_template = chatSummaryPrompt,
        summary_frequency = 4
    ),
)

In [4]:
# UI
import gradio as gr

def agentChat(message, history):
    agentResponse = agentQuery.run(message)
    agentResponse = agentResponse["answers"][0].answer
    
    for i in range(len(agentResponse)):
        time.sleep(0.03)
        yield agentResponse[: i+1]

# TODO Add a button to add your system prompt
# TODO Add a button to upload your docs (pdf's)
agentChat = gr.ChatInterface(
    fn = agentChat,
)

agentChat.queue()
agentChat.launch()